In [1]:
# %%capture
# %pip install unsloth
# # Also get the latest nightly Unsloth!
# %pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# # Install Flash Attention 2 for softcapping support
# import torch
# if torch.cuda.get_device_capability()[0] >= 8:
#     %pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [1]:
from unsloth import FastLanguageModel
import torch

# 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# max_seq_length = 4096  # 공식 튜토리얼은 2048
max_seq_length = 512
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit", # 공식 튜토리얼
    # model_name="unsloth/gemma-2-9b",  # 모델 이름을 설정합니다.
    model_name="yanolja/EEVE-Korean-10.8B-v1.0",
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: OpenAI failed to import - ignoring for now.
==((====))==  Unsloth 2024.11.5: Fast Llama patching. Transformers = 4.46.2.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.988 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [2]:
tokenizer

LlamaTokenizerFast(name_or_path='yanolja/EEVE-Korean-10.8B-v1.0', vocab_size=40960, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 2, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 4,
    lora_dropout = 0.01, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.01.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.11.5 patched 48 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs       = examples["input"]
#     outputs      = examples["output"]
#     texts = []
#     for instruction, input, output in zip(instructions, inputs, outputs):
#         # Must add EOS_TOKEN, otherwise your generation will go on forever!
#         text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text" : texts, }
# pass

# from datasets import load_dataset
# # dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# # dataset = dataset.map(formatting_prompts_func, batched = True,)

In [5]:
prompt_input_template = """아래는 작업을 설명하는 지시사항과 추가 정보를 제공하는 입력이 짝으로 구성됩니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 입력:
{input}

### 응답:"""


prompt_no_input_template = """아래는 작업을 설명하는 지시사항입니다. 이에 대한 적절한 응답을 작성해주세요.

### 지시사항:
{instruction}

### 응답:"""

def generate_prompt(data_point):
    instruction = data_point["instruction"]
    input = data_point["input"]
    label = data_point["output"]

    if input:
        res = prompt_input_template.format(instruction=instruction, input=input)
    else:
        res = prompt_no_input_template.format(instruction=instruction)

    if label:
        res = f"{res}{label}<|im_end|>" # eos_token을 마지막에 추가

    data_point['text'] = res

    return data_point

In [ ]:
import json
from datasets import Dataset
import pandas as pd

with open("data/instruction_dataset/qa_dataset_introduction_gpt4o_tourLLM_0.5K.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

dataset2 = Dataset.from_pandas(pd.DataFrame(json_data))

dataset2 = dataset2.rename_column("question", "instruction")
dataset2 = dataset2.rename_column("answer", "output")

dataset2 = dataset2.remove_columns(["tourism_attr_name"])

dataset2

def combine_columns(example):
    instruction = example['instruction']
    # input_text = example['input'] if example['input'] else ""  # input이 비어있을 경우를 처리
    input_text = ""
    output = example['output']

    # 원하는 형식으로 텍스트를 결합
    combined_text = (f"Below is an instruction that describes a task, paired with an input that provides further context. "
                     f"Write a response that appropriately completes the request.\n\n"
                     f"### Instruction:\n{instruction}\n\n"
                     f"### Input:\n{input_text}\n\n"
                     f"### Response:\n{output}</s>")

    example["text"] = combined_text
    example['input'] = ""
    return example


dataset2 = dataset2.map(combine_columns)
# dataset2 = dataset2.map(formatting_prompts_func, batched = True, )

# remove_column_keys = dataset2.features.keys() # 기존 컬럼(instruction, output 등) 제거
dataset_cvted = dataset2.shuffle().map(generate_prompt)

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

In [7]:
def tokenize_function(examples):
  outputs = tokenizer(examples["text"], truncation=True, max_length=512)
  return outputs

# remove_column_keys = dataset_cvted.features.keys()
dataset_tokenized = dataset_cvted.map(tokenize_function, batched=True)

Map:   0%|          | 0/246 [00:00<?, ? examples/s]

In [8]:
def collate_fn(examples):
    examples_batch = tokenizer.pad(examples, padding='longest', return_tensors='pt')
    examples_batch['labels'] = examples_batch['input_ids'] # 모델 학습 평가를 위한 loss 계산을 위해 입력 토큰을 레이블로 사용
    return examples_batch

In [9]:
tokenizer

LlamaTokenizerFast(name_or_path='yanolja/EEVE-Korean-10.8B-v1.0', vocab_size=40960, model_max_length=4096, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token if tokenizer.pad_token is None else tokenizer.pad_token

eval_step = 20

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    # train_dataset = dataset2,
    train_dataset = dataset_tokenized,
    data_collator = collate_fn,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        num_train_epochs= 30,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 8,
        warmup_steps = 1,
        # max_steps = 500, # Set num_train_epochs = 1 for full training runs
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        # fp16 = True,
        # bf16 = False,
        logging_steps = 20,
        # optim = "adamw_8bit",
        optim = "paged_adamw_8bit",
        save_strategy="steps",
        save_steps=eval_step,
        save_total_limit=2,
        weight_decay = 0.01,
        # lr_scheduler_type = "linear",
        lr_scheduler_type = "cosine",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [11]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 246 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 8
\        /    Total batch size = 16 | Total steps = 450
 "-____-"     Number of trainable parameters = 2,555,904


Step,Training Loss
20,1.884300
40,1.211300
60,1.099900
80,1.028800
100,0.884100
120,0.799100
140,0.719900
160,0.598500
180,0.509700
200,0.443100


In [ ]:
from unsloth import FastLanguageModel
import torch

# 최대 시퀀스 길이를 설정합니다. 내부적으로 RoPE 스케일링을 자동으로 지원합니다!
# max_seq_length = 4096  # 공식 튜토리얼은 2048
max_seq_length = 512
# 자동 감지를 위해 None을 사용합니다. Tesla T4, V100은 Float16, Ampere+는 Bfloat16을 사용하세요.
dtype = None
# 메모리 사용량을 줄이기 위해 4bit 양자화를 사용합니다. False일 수도 있습니다.
load_in_4bit = True

# 4배 빠른 다운로드와 메모리 부족 문제를 방지하기 위해 지원하는 4bit 사전 양자화 모델입니다.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit",  # Gemma 7b의 Instruct 버전
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit",  # Gemma 2b의 Instruct 버전
    "unsloth/llama-3-8b-bnb-4bit",  # Llama-3 8B
]  # 더 많은 모델은 https://huggingface.co/unsloth 에서 확인할 수 있습니다.

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-bnb-4bit", # 공식 튜토리얼
    # model_name="unsloth/gemma-2-9b",  # 모델 이름을 설정합니다.
    # model_name="yanolja/EEVE-Korean-10.8B-v1.0",
    model_name="/home/hwang/projects/yanolja/src/modeling/outputs/checkpoint-225",
    max_seq_length=max_seq_length,  # 최대 시퀀스 길이를 설정합니다.
    dtype=dtype,  # 데이터 타입을 설정합니다.
    load_in_4bit=load_in_4bit,  # 4bit 양자화 로드 여부를 설정합니다.
    # token = "hf_...", # 게이트된 모델을 사용하는 경우 토큰을 사용하세요. 예: meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.46.3.
   \\   /|    GPU: NVIDIA L40S. Max memory: 44.527 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.1+cu124. CUDA = 8.9. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '
<string>:204: SyntaxWarning: invalid escape sequence '\ '
<string>:205: SyntaxWarning: invalid escape sequence '\_'
<string>:206: SyntaxWarning: invalid escape sequence '\ '


OSError: could not get source code

In [39]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    prompt_input_template.format(
        instruction = "여행지 추천해줘.", # instruction
        # input = "대구에서 가볼만 한 수목원 알려줘.", # input
        input = "서울의 겨울 명소를 알려줘."
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 256, use_cache = True)

response = tokenizer.batch_decode(outputs)

In [ ]:
response[0].split("### 응답:")[1]

"서울의 겨울 명소 중 하나는 바로 대방역 1번 출구에서 시작하는 '한양도성 걷기 코스'입니다. 이 코스에서는 눈 덮인 설경 속에서 역사적인 벽을 따라 걷는 특별한 경험을 할 수 있습니다. 또한, 서울숲은 겨울에 특히 아름다운데, 다양한 나무들이 보여주는 겨울 숲의 모습은 고요하면서도 강렬한 인상을 남깁니다. 여의도 한양도선 시점은 겨울에 특히 인기 있는 장소로, 여의도역 4번 출구에서 접근이 쉽고, 서울애플피어와 연결되어 있어 도시 속에서 즐기는 겨울 나들이에 안성맞춤입니다. 마지막으로, 동대문역사문화공원은 동대문역 1, 2번 출구에서 접근 가능한 장소로, 이곳에서는 서울의 역사와 문화를 깊이 있게 체험할 수 있습니다. 이러한 장소들은 겨울에 특히 매력적인 선택을 제공합니다.<|im_end|>"

In [ ]:
## 학습한 데이터 양과 질이 부족함. 당연히 Align이 제대로 되지 않음. 2차년도부터 개선해 나갈 예정.